In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# !pip install git+https://github.com/m-bain/whisperx.git@v3
!pip install ultralytics
!pip install -q gradio
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.110 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 23.4/78.2 GB disk)


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
# import whisperx
import os
from moviepy.editor import *
from google.colab.patches import cv2_imshow
import moviepy.editor as mp
import gradio as gr

In [ ]:

# Step 1: Input is a video
def process_video(input_video):

  # Step 2: Extract frames
  video = cv2.VideoCapture(input_video)
  fps = video.get(cv2.CAP_PROP_FPS)
  frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
  frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
  output_video = "/content/gdrive/My Drive/demoM/test_images/output_video.mp4"
  output_writer = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*"mp4v"), fps, (frame_width, frame_height))

  # Step 3: Pass each frame to a model to detect objects
  model = YOLO("/content/gdrive/My Drive/demoM/trainedModel/bestV1.pt")

  # Step 4: Extract audio from the input video
  video_clip = mp.VideoFileClip(input_video)
  audio_clip = video_clip.audio
  audio_clip.write_audiofile("temp_audio.wav")

  while True:
      # Read the next frame
      try:
        ret, frame = video.read()
        if not ret:
            break
        # Step 5: Get the mask of the object and blur the mask
        results = model.predict(source=frame, conf=0.5)
        objects_mask = results[0].masks.xy
        mask_ = np.zeros_like(frame)
        for mask in objects_mask:
          vertices = mask.reshape(mask.shape[0], mask.shape[1] // 2, 2)
          cv2.fillPoly(mask_, np.int32([vertices]), (255, 255, 255))
          #blurred_region1 = cv2.blur(frame, (20, 20), 10)
          blurred_region2 = cv2.GaussianBlur(frame, (53, 53), 30)
          output_frame = np.where(mask_ != 0, blurred_region2, frame)
        output_writer.write(output_frame)
      except Exception as e:
        output_writer.write(frame)

  # Release the video capture and writer, and close windows
  video.release()
  output_writer.release()

  # Step 8: Merge the output video with the audio using moviepy
  final_output = mp.VideoFileClip(output_video)
  final_output = final_output.set_audio(mp.AudioFileClip("temp_audio.wav"))
  final_output.write_videofile("/content/gdrive/My Drive/demoM/test_images/final_output_video.mp4", codec="libx264", audio_codec="aac", remove_temp=True)
  return "/content/gdrive/My Drive/demoM/test_images/final_output_video.mp4"


# Define the input and output components
input_video = gr.inputs.Video(type="mp4", label="Input Video")
output_video = gr.outputs.File(label="Processed Video")

# Create the Gradio interface
interface = gr.Interface(fn=process_video, inputs=input_video, outputs=output_video)

# Launch the interface
interface.launch(share=True)


  warnings.warn(

  super().__init__(format=type, source=source, label=label, optional=optional)

  warnings.warn(



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://81e964173c003c49d1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
